In [3]:
import pandas as pd
from utils.offloading import task_offloading
from utils.objects import *

In [2]:
rsu = pd.read_csv('data/rsu.csv')
es = pd.read_csv('data/es.csv')
tasks = pd.read_csv('data/tasks.csv')

In [4]:
rsu_bank = populate_rsus(rsu)
es_bank = populate_ess(es)
tasks_bank = populate_tasks(tasks)

In [6]:
import numpy as np

# Define the parameters of the ACO algorithm
NUM_ANTS = 20 # number of ants in the colony
NUM_ITERATIONS = 100 # number of iterations to run the algorithm
EVAPORATION_RATE = 0.5 # rate at which pheromone evaporates
ALPHA = 1.0 # importance of pheromone level in selecting next node
BETA = 1.0 # importance of heuristic information in selecting next node
Q = 1.0 # pheromone deposit constant
INITIAL_PHEROMONE = 0.1 # initial pheromone level on each edge

# Define the problem-specific parameters
NUM_TASKS = len(tasks_bank) # number of tasks in the network
NUM_RSUS = len(rsu_bank) # number of RSUs in the network
#MAX_COMPUTATION_TIME = 100 # maximum computation time for a task
#MAX_MIGRATION_TIME = 50 # maximum migration time for a task
#MAX_DTR = 10 # maximum data transfer rate for an RSU
#TASK_LENGTHS = np.random.randint(1, 11, NUM_TASKS) # length of each task
#TASK_FILE_SIZES = np.random.randint(1, 11, NUM_TASKS) # size of the file associated with each task
#RSU_DTRS = np.random.randint(1, MAX_DTR+1, NUM_RSUS) # data transfer rates of each RSU


In [7]:
# Define the adjacency matrix for the network
network = np.random.rand(NUM_RSUS, NUM_RSUS)
network = np.tril(network) + np.tril(network, -1).T # make the matrix symmetric
network = network / np.sum(network, axis=1)[:, np.newaxis] # normalize each row to sum to 1

In [ ]:
# Define the pheromone matrix and heuristic information matrix
pheromones = np.full((NUM_RSUS, NUM_RSUS), INITIAL_PHEROMONE)
heuristics = np.empty((NUM_RSUS, NUM_RSUS))
for i in range(NUM_RSUS):
    for j in range(NUM_RSUS):
        if i == j:
            heuristics[i,j] = 0
        else:
            heuristics[i,j] = 1 / network[i,j]

def compute_fitness(solution):
    final_tasks = task_offloading(network=solution, tasks_bank=tasks_bank)
    # get the maximum of MIGRATION_HISTORY and COMPUTATION_HISTORY for each task
    max_migration = max([max(task.MIGRATION_HISTORY) for task in final_tasks])
    max_computation = max([max(task.COMPUTATION_HISTORY) for task in final_tasks])

    return max_migration, max_computation

# Define a function to update the pheromone matrix
def update_pheromones(pheromones, ants, obj_func, evaporation_rate, alpha, beta):
    # Update pheromone levels on each edge
    for i, j in np.ndindex(pheromones.shape):
        # Calculate the amount of pheromone deposited on the edge
        pheromone_delta = 0
        for ant in ants:
            if (i, j) in ant.path:
                # Calculate the objective function value for the ant's path
                path_obj_func_val = obj_func(ant.path)
                # Calculate the amount of pheromone to be deposited on the edge
                ant_pheromone_delta = 1 / path_obj_func_val
                pheromone_delta += ant_pheromone_delta

        # Update the pheromone level on the edge
        pheromones[i, j] = (1 - evaporation_rate) * pheromones[i, j] + evaporation_rate * pheromone_delta

    # Apply the pheromone limits (optional)
    # pheromones = np.clip(pheromones, pheromone_min, pheromone_max)

    return pheromones
